In [17]:
import nltk
import string
from unidecode import unidecode

nltk.download('stopwords')


def processar_texto_ingredientes(texto_ingredientes):
    texto_ingredientes = texto_ingredientes.lower()
    ingredientes_tokens = nltk.tokenize.word_tokenize(texto_ingredientes, language='portuguese')
    stopwords = set(nltk.corpus.stopwords.words('portuguese'))
    pontuacoes = set(string.punctuation)
    stopwords_and_pontuacoes = stopwords.union(pontuacoes)
    ingredientes_tokens = [unidecode(x) for x in ingredientes_tokens if x not in stopwords_and_pontuacoes]
    return ingredientes_tokens


ingredientes_torrone = 'Xarope de glicose, açúcar, amendoim torrado, clara de ovo pasteurizada, farinha de arroz, óleo de soja, aromatizante extrato natural de baunilha e emulsificante lecitina de soja.'

print(processar_texto_ingredientes(ingredientes_torrone))

['xarope', 'glicose', 'acucar', 'amendoim', 'torrado', 'clara', 'ovo', 'pasteurizada', 'farinha', 'arroz', 'oleo', 'soja', 'aromatizante', 'extrato', 'natural', 'baunilha', 'emulsificante', 'lecitina', 'soja', 'contem', 'gluten']


[nltk_data] Downloading package stopwords to /home/chris/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
